# 🛰️ KLYMO ASCENT ML - Satellite Super-Resolution

**Geospatial Super-Resolution Pipeline with Hallucination Prevention**

This notebook demonstrates:
- 8x super-resolution (10m → 1.25m/pixel)
- ESRGAN with Geospatial Attention Module (GAM)
- Spatial Consistency Loss for hallucination prevention
- Quality metrics: PSNR, SSIM, Edge Coherence

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/klymo-ascent-ml/blob/main/notebooks/klymo_inference.ipynb)

## 1. Setup Environment

In [ ]:
# Install dependencies (run once)
!pip install torch torchvision --quiet
!pip install opencv-python pillow scikit-image matplotlib tqdm --quiet

# Clone repository (if running on Colab)
import os
if not os.path.exists('klymo-ascent-ml'):
    !git clone https://github.com/your-repo/klymo-ascent-ml.git
    %cd klymo-ascent-ml
else:
    %cd klymo-ascent-ml

In [ ]:
# Import libraries
import sys
sys.path.insert(0, 'src')

import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import display
import cv2

# Check GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Load Model

In [ ]:
from model import create_generator

# Create generator
generator = create_generator(
    use_gam=False,  # Use RGB-only model for simplicity
    in_channels=3,
    num_rrdb_blocks=23,
    scale_factor=8
).to(device)

# Load checkpoint (if available)
checkpoint_path = 'checkpoints/best_model.pth'

import os
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    generator.load_state_dict(checkpoint['generator'])
    print(f"✓ Loaded checkpoint: {checkpoint_path}")
    if 'psnr' in checkpoint:
        print(f"  Checkpoint PSNR: {checkpoint['psnr']:.2f} dB")
else:
    print("⚠ No checkpoint found. Using untrained model for demo.")

generator.eval()
print(f"Model parameters: {sum(p.numel() for p in generator.parameters()):,}")

## 3. Create Sample Image

In [ ]:
def create_sample_satellite_image(size=64):
    """Create a synthetic satellite-like image."""
    np.random.seed(42)
    image = np.random.rand(size, size, 3) * 0.3 + 0.2
    
    # Add buildings
    for _ in range(np.random.randint(5, 15)):
        x = np.random.randint(0, size - 15)
        y = np.random.randint(0, size - 15)
        w = np.random.randint(5, 15)
        h = np.random.randint(5, 15)
        color = np.random.rand(3) * 0.3 + 0.4
        image[y:y+h, x:x+w] = color
    
    # Add roads
    for _ in range(np.random.randint(2, 4)):
        if np.random.rand() > 0.5:
            y = np.random.randint(0, size)
            image[max(0, y-1):min(size, y+1), :] = [0.3, 0.3, 0.35]
        else:
            x = np.random.randint(0, size)
            image[:, max(0, x-1):min(size, x+1)] = [0.3, 0.3, 0.35]
    
    # Add vegetation
    for _ in range(np.random.randint(2, 5)):
        x = np.random.randint(0, size - 10)
        y = np.random.randint(0, size - 10)
        r = np.random.randint(3, 7)
        yy, xx = np.ogrid[:size, :size]
        mask = ((xx - x - r) ** 2 + (yy - y - r) ** 2) < r ** 2
        image[mask] = [0.2, 0.5 + np.random.rand() * 0.2, 0.15]
    
    return (image * 255).astype(np.uint8)

# Create sample LR image
lr_image = create_sample_satellite_image(64)

plt.figure(figsize=(6, 6))
plt.imshow(lr_image)
plt.title(f'Low Resolution Input ({lr_image.shape[1]}x{lr_image.shape[0]})')
plt.axis('off')
plt.show()

## 4. Super-Resolution Inference

In [ ]:
def enhance_image(model, lr_image, device='cuda'):
    """Apply super-resolution to an image."""
    # Preprocess
    lr_tensor = torch.from_numpy(lr_image).float().permute(2, 0, 1).unsqueeze(0) / 255.0
    lr_tensor = lr_tensor.to(device)
    
    # Inference
    with torch.no_grad():
        sr_tensor = model(lr_tensor)
    
    # Postprocess
    sr_tensor = torch.clamp(sr_tensor, 0, 1)
    sr_image = sr_tensor.squeeze(0).permute(1, 2, 0).cpu().numpy()
    sr_image = (sr_image * 255).astype(np.uint8)
    
    return sr_image

# Generate SR output
print("Generating super-resolved image...")
sr_image = enhance_image(generator, lr_image, device)
print(f"SR output shape: {sr_image.shape}")

# Generate bicubic baseline
lr_tensor = torch.from_numpy(lr_image).float().permute(2, 0, 1).unsqueeze(0) / 255.0
bicubic_tensor = F.interpolate(lr_tensor, scale_factor=8, mode='bicubic', align_corners=False)
bicubic_image = (bicubic_tensor.squeeze(0).permute(1, 2, 0).numpy() * 255).astype(np.uint8)
bicubic_image = np.clip(bicubic_image, 0, 255).astype(np.uint8)

## 5. Visual Comparison

In [ ]:
# Display comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# LR (upscaled for display)
lr_display = cv2.resize(lr_image, (512, 512), interpolation=cv2.INTER_NEAREST)
axes[0].imshow(lr_display)
axes[0].set_title(f'Low Resolution\n({lr_image.shape[1]}x{lr_image.shape[0]} → display)')
axes[0].axis('off')

# Bicubic
axes[1].imshow(bicubic_image)
axes[1].set_title(f'Bicubic 8x\n({bicubic_image.shape[1]}x{bicubic_image.shape[0]})')
axes[1].axis('off')

# SR
axes[2].imshow(sr_image)
axes[2].set_title(f'Super-Resolved 8x\n({sr_image.shape[1]}x{sr_image.shape[0]})')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 6. Zoom Comparison

In [ ]:
# Zoom into a region
zoom_x, zoom_y = 128, 128
zoom_size = 128

bicubic_crop = bicubic_image[zoom_y:zoom_y+zoom_size, zoom_x:zoom_x+zoom_size]
sr_crop = sr_image[zoom_y:zoom_y+zoom_size, zoom_x:zoom_x+zoom_size]

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(bicubic_crop)
axes[0].set_title('Bicubic (Zoomed 4x)')
axes[0].axis('off')

axes[1].imshow(sr_crop)
axes[1].set_title('Super-Resolved (Zoomed 4x)')
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 7. Difference Map (Hallucination Detection)

In [ ]:
# Compute difference map
diff = np.abs(sr_image.astype(np.float32) - bicubic_image.astype(np.float32))
diff_gray = np.mean(diff, axis=2)
diff_normalized = (diff_gray / diff_gray.max() * 255).astype(np.uint8)

# Apply colormap
diff_heatmap = cv2.applyColorMap(diff_normalized, cv2.COLORMAP_JET)
diff_heatmap = cv2.cvtColor(diff_heatmap, cv2.COLOR_BGR2RGB)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(sr_image)
axes[0].set_title('Super-Resolved Output')
axes[0].axis('off')

axes[1].imshow(diff_heatmap)
axes[1].set_title('Difference Map (SR vs Bicubic)\nHot = High difference')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("The difference map shows where SR added details.")
print("High values in unexpected areas may indicate hallucination.")

## 8. Quality Metrics

In [ ]:
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# PSNR
psnr = peak_signal_noise_ratio(bicubic_image, sr_image, data_range=255)
print(f"PSNR: {psnr:.2f} dB")

# SSIM
ssim = structural_similarity(bicubic_image, sr_image, data_range=255, channel_axis=2)
print(f"SSIM: {ssim:.4f}")

# Edge Coherence
sr_gray = cv2.cvtColor(sr_image, cv2.COLOR_RGB2GRAY)
bicubic_gray = cv2.cvtColor(bicubic_image, cv2.COLOR_RGB2GRAY)

sr_edges = cv2.Canny(sr_gray, 50, 150).flatten().astype(float)
bicubic_edges = cv2.Canny(bicubic_gray, 50, 150).flatten().astype(float)

if sr_edges.std() > 0 and bicubic_edges.std() > 0:
    edge_coherence = np.corrcoef(sr_edges, bicubic_edges)[0, 1]
    edge_coherence = max(0, edge_coherence) if not np.isnan(edge_coherence) else 0
else:
    edge_coherence = 1.0

print(f"Edge Coherence: {edge_coherence:.4f}")

# Interpretation
print("\n--- Interpretation ---")
print(f"PSNR: {'✓ Good' if psnr > 25 else '⚠ Low'} (target: >28 dB)")
print(f"SSIM: {'✓ Good' if ssim > 0.85 else '⚠ Fair' if ssim > 0.8 else '✗ Low'} (target: >0.85)")
print(f"Edge Coherence: {'✓ No major hallucination' if edge_coherence > 0.85 else '⚠ Check for hallucination'} (target: >0.90)")

## 9. Edge Overlay Visualization

In [ ]:
def create_edge_overlay(image, edge_color=(255, 0, 0), alpha=0.5):
    """Create edge overlay visualization."""
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    
    overlay = image.copy()
    overlay[edges > 0] = edge_color
    
    result = cv2.addWeighted(image, 1 - alpha, overlay, alpha, 0)
    return result

# Create edge overlays
bicubic_edges_overlay = create_edge_overlay(bicubic_image)
sr_edges_overlay = create_edge_overlay(sr_image)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(bicubic_edges_overlay)
axes[0].set_title('Bicubic - Edge Overlay')
axes[0].axis('off')

axes[1].imshow(sr_edges_overlay)
axes[1].set_title('Super-Resolved - Edge Overlay')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("Edge overlay shows detected edges in red.")
print("SR should have similar edge locations as bicubic (no hallucinated edges).")

## 10. Upload Your Own Image

In [ ]:
# Upload image (works in Colab/Jupyter)
try:
    from google.colab import files
    print("Upload a low-resolution satellite image:")
    uploaded = files.upload()
    
    if uploaded:
        filename = list(uploaded.keys())[0]
        custom_image = np.array(Image.open(filename).convert('RGB'))
        
        # Resize if too large
        max_size = 128
        if max(custom_image.shape[:2]) > max_size:
            scale = max_size / max(custom_image.shape[:2])
            new_size = (int(custom_image.shape[1] * scale), int(custom_image.shape[0] * scale))
            custom_image = cv2.resize(custom_image, new_size)
        
        print(f"Image loaded: {custom_image.shape}")
        
        # Enhance
        custom_sr = enhance_image(generator, custom_image, device)
        
        # Display
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        axes[0].imshow(custom_image)
        axes[0].set_title('Input')
        axes[0].axis('off')
        axes[1].imshow(custom_sr)
        axes[1].set_title('Super-Resolved 8x')
        axes[1].axis('off')
        plt.show()
        
except ImportError:
    print("File upload not available in this environment.")
    print("To use your own image, place it in the working directory and load with:")
    print("  custom_image = np.array(Image.open('your_image.png').convert('RGB'))")

## 11. Summary

This notebook demonstrated the KLYMO ASCENT ML super-resolution pipeline:

1. **8x Super-Resolution**: 10m → 1.25m/pixel
2. **ESRGAN Architecture**: Enhanced with RRDB blocks
3. **Hallucination Prevention**: Spatial Consistency Loss + Edge Coherence metric
4. **Quality Metrics**: PSNR, SSIM, Edge Coherence

### Key Takeaways:
- **PSNR >28 dB**: Good reconstruction quality
- **SSIM >0.85**: Good perceptual similarity
- **Edge Coherence >0.90**: No major hallucination

### Next Steps:
- Train on real Sentinel-2/WorldView pairs
- Enable Geospatial Attention Module (GAM)
- Deploy to Hugging Face Spaces